# Stateful detection of black-box adversarial attack
Example for PyTorch library

## Setup

In [1]:
import numpy as np
import random 
np.random.seed(2137)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from scipy.ndimage import rotate

from art.estimators.encoding import PyTorchEncoder
from art.attacks.evasion import HopSkipJump
from art.estimators.classification import PyTorchClassifier
from art.defences.detector.evasion.black_box import MemoryQueue, BlackBoxDetector, NearestNeighborsWrapper, torch_contrastive_loss
import matplotlib.pyplot as plt

In [ ]:
device = "cpu"
batch_size = 64
ds_kwargs = {
        'batch_size'  : batch_size
}

## Get data

In [3]:
def get_dataset():
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    train_ds = datasets.MNIST('./data', train=True, download=True, transform=transform)
    test_ds = datasets.MNIST('./data', train=False, transform=transform)
    return train_ds, test_ds

In [ ]:
train_ds, test_ds = get_dataset()

In [7]:
def apply_random_uniform_noise(X: np.ndarray, r: float = 0.5) -> np.ndarray:
    return X + np.random.uniform(-r, r, size=X.shape)

def apply_random_gauss_noise(X: np.ndarray, r: float = 0.5) -> np.ndarray:
    return X + r * np.random.normal(0, 1, size=X.shape)

def apply_random_rotation(X: np.ndarray, r: int = 10) -> np.ndarray:
    return rotate(X, angle=np.random.randint(0 , r), reshape=False, axes=(-1, -2))

def apply_random_translation(X: np.ndarray, r: int = 5) -> np.ndarray:
    c_h = np.random.randint(-r, r)
    c_v = np.random.randint(-r, r)
    return np.roll(np.roll(X, c_v, axis=-1), c_h, axis=-2)
    
def apply_scale_on_pixel(X: np.ndarray, r: float = 0.5) -> np.ndarray:
    return X * random.uniform(1 - r, 1 + r)

def apply_brightness_adjustment(X: np.ndarray, r: float = 0.5) -> np.ndarray:
    return X + np.random.uniform(-r, r)

ALL_TRANSFORMATIONS = [
    apply_random_gauss_noise,
    apply_brightness_adjustment,
    apply_random_translation,
    apply_random_uniform_noise,
    apply_random_rotation,
    apply_scale_on_pixel
]

def apply_random_transformation(X: np.ndarray) -> np.ndarray:
    fn = random.choice(ALL_TRANSFORMATIONS)
    return fn(X)

In [19]:
def get_encoder_dataset_loader(ds):
    X = None
    Y = None
    labels = ds.train_labels
    for i in range(10):
        indexes = (labels == i).nonzero().flatten()
        ith_data = ds1.data[indexes].numpy()
        indexes = (labels != i).nonzero().flatten()
        non_ith_data = ds1.data[indexes]
        transformed = apply_random_transformation(ith_data)
        if X is None:
            X = np.stack([ith_data, transformed], axis=1)
        else:
            X = np.concatenate((X, np.stack([ith_data, transformed], axis=1)))
        rand_indexes = np.random.choice(len(non_ith_data), len(ith_data))
        transformed = apply_random_transformation(non_ith_data[rand_indexes])
        if Y is None:
            Y = np.stack([ith_data, transformed], axis=1)
        else:
            Y = np.concatenate((Y, np.stack([ith_data, transformed], axis=1)))
    shuff = np.random.permutation(len(X))
    X = X[shuff]
    Y = Y[shuff]
    X, Y = X / 255., Y / 255.
    X, Y = torch.Tensor(X), torch.Tensor(Y)
    contrastive_ds = torch.utils.data.TensorDataset(X, Y)
    ds_loader = torch.utils.data.DataLoader(contrastive_ds, **ds_kwargs)
    return ds_loader

In [ ]:
encoder_ds_loader = get_encoder_dataset_loader(train_ds)

## Prepare encoder

In [228]:
def train_encoder(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data  = torch.unsqueeze(data, dim=-3).to(device)
        labels  = torch.unsqueeze(labels, dim=-3).to(device)
        x_i, x_p = data[:, 0, ...], data[:, 1, ...]
        x_j, x_n = labels[:, 0, ...], labels[:, 1, ...]
        optimizer.zero_grad()
        e_i, e_j = model(x_i), model(x_j)
        e_p, e_n = model(x_p), model(x_n)
        loss = torch_contrastive_loss(e_i, e_p, e_j, e_n)
        loss.backward()
        optimizer.step()
        if ( batch_idx * len(data) ) % 1000 == 0:
            print('Train Epoch: {:3} [{:5}/{} ({:.0f}%)]\tLoss: {:5.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))

In [229]:
class Encoder(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 64)
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
encoder_model = Encoder()

optimizer = optim.Adam(encoder_model.parameters())

epochs = 10 

for epoch in range(1, epochs + 1):
    train_encoder(encoder_model, device, encoder_ds_loader, optimizer, epoch)

Train Epoch:   1 [    0/60000 (0%)]	Loss: 6.762132
Train Epoch:   1 [ 8000/60000 (13%)]	Loss: 0.660766
Train Epoch:   1 [16000/60000 (27%)]	Loss: 0.703214
Train Epoch:   1 [24000/60000 (40%)]	Loss: 1.357955
Train Epoch:   1 [32000/60000 (53%)]	Loss: 0.466298
Train Epoch:   1 [40000/60000 (67%)]	Loss: 0.327619
Train Epoch:   1 [48000/60000 (80%)]	Loss: 0.436096
Train Epoch:   1 [56000/60000 (93%)]	Loss: 0.773699
Train Epoch: 1 	Loss: 0.607402
Train Epoch:   2 [    0/60000 (0%)]	Loss: 0.889977
Train Epoch:   2 [ 8000/60000 (13%)]	Loss: 0.390618
Train Epoch:   2 [16000/60000 (27%)]	Loss: 0.138890
Train Epoch:   2 [24000/60000 (40%)]	Loss: 0.026890
Train Epoch:   2 [32000/60000 (53%)]	Loss: 0.838529
Train Epoch:   2 [40000/60000 (67%)]	Loss: 0.057897
Train Epoch:   2 [48000/60000 (80%)]	Loss: 0.041418
Train Epoch:   2 [56000/60000 (93%)]	Loss: 0.105490
Train Epoch: 2 	Loss: 0.024077
Train Epoch:   3 [    0/60000 (0%)]	Loss: 0.026329
Train Epoch:   3 [ 8000/60000 (13%)]	Loss: 0.121882
Train

## Prepare classifier

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.fc2(x)
        x = torch.nn.functional.log_softmax(x)
        return x

classifier_model = Classifier()
optimizer = torch.optim.SGD(classifier_model.parameters(), lr=1e-4)

epochs = 5

def train_classifier_step(epoch):
    classifier_model.train()
    for batch_idx, (data, target) in enumerate(train_ds):
        optimizer.zero_grad()
        output = classifier_model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))
        
for i in range(1, epochs + 1):
    train_classifier_step(i)

In [ ]:
classifier = PyTorchClassifier(
    model=classifier_model,
    loss=torch.nn.functional.nll_loss,
    input_shape=(1, 28, 28),
    nb_classes=10
)

Classifier simple validation:

In [ ]:
example = test_ds.data[100].numpy()
example = example.reshape(-1, 28, 28)
plt.imshow(example[0], cmap="gray")

In [ ]:
np.argmax(classifier.predict(example))

## Detection

In [ ]:
def get_detector_env(encoder_model, ds, k, threshold):
    encoder = PyTorchEncoder(model=model, input_shape=(1, 28, 28), encoding_length=256)
    queue = MemoryQueue(max_size=1000, element_shape=(64,))
    for data in ds.data:
        data = np.reshape(data.numpy(), (1, 28, 28))
        encoded_data = encoder.predict(data).numpy()
        queue.extend(encoded_data)
        if len(queue.get_memory()) > 900:
            break
    knn = NearestNeighborsWrapper(k_neighbors=k, memory_queue=queue)
    knn.update_memory()
    detector = BlackBoxDetector(
        similarity_encoder=encoder,
        memory_queue=queue,
        knn=knn,
        detection_threshold=threshold
    )
    return detector, encoder, queue, knn

### Normal user action

Here user is just doing the classification

In [ ]:
detector, encoder, queue, knn = get_detector_env(encoder_model, train_ds, 20, 1.0)

In [ ]:
for idx, data in enumerate(test_ds.data[10:]):
    data = np.reshape(data.numpy(), [1, 28, 28])
    is_attack, distance = detector.scan(data)
    if idx % 100 == 0:
        print(f"Is attack detected: {is_attack}, adv example mean k-distance: {distance}")
    encoded_data = encoder.predict(data)
    queue.extend(encoded_data)
    knn.update_memory()
    if is_attack:
        print(idx)
        print("Attack detected!")
        break
        
    if idx == 2000:
        print("Normal action")
        break

As we can see attack was not detected

### Adversarial attack

Here user is trying to attack the classifier

In [ ]:
detector, encoder, queue, knn = get_detector_env(encoder_model, train_ds, 20, 1.0)

In [204]:
target = test_ds.data[0].numpy()
target = target.reshape(-1, 28, 28)

In [221]:
attack = HopSkipJump(
    classifier=classifier,
    targeted=False,
    max_iter=0,
    max_eval=1,
    init_eval=1
)

In [227]:
iter_step = 10
x_adv = None
for i in range(2000):
    x_adv = attack.generate(x=np.array([target]), x_adv_init=x_adv, resume=True)
    
    print(f"L2 error {np.linalg.norm(x_adv[0] - target)}, predicted class {np.argmax(classifier.predict(x_adv))}")
    attack.max_iter = iter_step
    
    is_detected, distance = detector.scan(x_adv)
    print(f"Is attack detected: {is_detected}, adv example mean k-distance: {distance}")
    encoded_adv = encoder.predict(x_adv)
    queue.extend(encoded_adv)
    knn.update_memory()
    if is_detected:
        print(f"Attack detected at iteration {i}!")
        break

/Users/twardy/Code/studia/inzynierka/adversarial-robustness-toolbox/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


HopSkipJump:   0%|          | 0/1 [00:00<?, ?it/s]

L2 error 1979.202392578125, predicted class 3
Is attack detected: False, adv example mean k-distance: 7.332423210144043


/Users/twardy/Code/studia/inzynierka/adversarial-robustness-toolbox/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


HopSkipJump:   0%|          | 0/1 [00:00<?, ?it/s]

...

HopSkipJump:   0%|          | 0/1 [00:00<?, ?it/s]

L2 error 694.8771362304688, predicted class 3
Is attack detected: False, adv example mean k-distance: 1.12043035030365


/Users/twardy/Code/studia/inzynierka/adversarial-robustness-toolbox/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


HopSkipJump:   0%|          | 0/1 [00:00<?, ?it/s]

L2 error 694.841552734375, predicted class 3
Is attack detected: True, adv example mean k-distance: 0.9886554479598999
363
